In [1]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from train_set import Training_set
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from numpy import mean
from numpy import std
import pandas as pd

In [2]:
chi = pd.read_csv("Chi_data.csv", index_col= 0)
X = chi
#Y has the last col with labels
y = Training_set.iloc[:, -1]
# split the dataset to train and test, the 2nd is for the non feature selection
Training_data = pd.merge(X, y, left_index=True, right_index=True)
print(Training_data)

X = Training_data.iloc[:, :-1]
#Y has the last col with labels
y = Training_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

           35076296-35282086  41062669-41447005  42161364-42296514  \
Array.129                  2                 -1                 -1   
Array.34                   0                  0                  0   
Array.67                  -1                  0                  0   
Array.24                   0                  1                  1   
Array.22                   0                  0                  0   
...                      ...                ...                ...   
Array.10                   2                 -1                 -1   
Array.123                  0                  0                  0   
Array.100                  0                  0                  0   
Array.134                  1                  1                  1   
Array.130                  2                 -1                 -1   

           85450052-85962613  41458670-41494331  84542006-85443011  \
Array.129                  0                 -1                  0   
Array.34           

In [3]:
# Normalize the features
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [4]:
# Split the generated data into training and testing sets
skf_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
actual_scores = []

# Define the parameter grid for the random forest classifier
param_grid = [{
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']}]

for train_index_inner, test_index_inner in skf_outer.split(X_train_norm, y_train):
    X_train_inner, X_test_inner = X_train_norm[train_index_inner], X_train_norm[test_index_inner]
    y_train_inner, y_test_inner = y_train.iloc[train_index_inner], y_train.iloc[test_index_inner]

    # parameter tuning for the random forest classifier using GridSearchCV
    knn = KNeighborsClassifier()

    # Inner cross-validation loop for hyperparameter tuning
    skf_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    # defining parameter range
    grid = GridSearchCV(knn, param_grid, cv=skf_inner, scoring='accuracy', return_train_score=False, verbose=1)

    grid_search= grid.fit(X_train_inner, y_train_inner)
    best_model = grid_search.best_estimator_
    yhat = best_model.predict(X_test_inner)
    acc = accuracy_score(y_test_inner, yhat)
    actual_scores.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, grid_search.best_score_, grid_search.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(actual_scores), std(actual_scores)))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.625, est=0.609, cfg={'n_neighbors': 3, 'weights': 'distance'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.750, est=0.610, cfg={'n_neighbors': 5, 'weights': 'uniform'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.562, est=0.641, cfg={'n_neighbors': 5, 'weights': 'distance'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.625, est=0.644, cfg={'n_neighbors': 7, 'weights': 'uniform'}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
>acc=0.625, est=0.658, cfg={'n_neighbors': 7, 'weights': 'distance'}
Accuracy: 0.637 (0.061)
